# CSE 569 

# PROJECT 1

# Feature Extraction, Density Estimation and Bayesian Classification

# Nitheese Thirumoorthy
# ASU ID : 1225417336

# Importing Libraries

In [1]:
#Importing the needed Libraries
import numpy as np
import pandas as pd
import os
import scipy.io
from scipy.stats import multivariate_normal

# Importing the Data

In [2]:
train_dataset = scipy.io.loadmat("train_data.mat")
ytrain=train_dataset['label'][0]
Xtrain=train_dataset['data']
print("Shape of Xtrain is :",Xtrain.shape)
print("Shape of ytrain is :",ytrain.shape)
print()


test_data = scipy.io.loadmat("test_data.mat")
ytest=test_data['label'][0]
Xtest=test_data['data']
print("Shape of Xtest is :",Xtest.shape)
print("Shape of ytest is :",ytest.shape)
print()

Shape of Xtrain is : (11548, 28, 28)
Shape of ytrain is : (11548,)

Shape of Xtest is : (2886, 28, 28)
Shape of ytest is : (2886,)



# Task 1 - Feature Extraction and Normalisation

In [3]:
#Funtion to Calculate the bright pixels to dark pixels ratio of an image.
def brighttodark(data,T):
    X=[]
    for img in data:
        bright,dark=0,0
        for i in range(len(img)):
            for j in range(len(img[0])):
                if img[i][j]>T:
                    bright+=1
                elif img[i][j]<=T:
                    dark+=1
        X.append(bright/dark)
    return np.array(X)
        

In [4]:
#Function to calculate the skewness of an image.
def skewness(data):
    X=[]
    for img in data:
        x_bar=np.mean(img)
        s=np.std(img)
        total=0
        for i in range(28):
            for j in range(28):
                total += ((img[i][j]-x_bar)**3)
    
        total=total/(28*28)
        X.append(total/(s**3))
    return np.array(X)
    

In [5]:
#Function to normalize our
def normalize(data,T):
    Xnorm=np.array([skewness(data),brighttodark(data,T)])
    mu=np.mean(Xnorm,axis=1)
    mu=np.reshape(mu,(2,1))
    Xnorm= Xnorm - mu
    std=np.std(Xnorm,axis=1)
    std=np.reshape(std,(2,1))
    Xnorm = Xnorm/std
    return Xnorm

# Density Estimation

In [6]:
#Calling the functions to do feature extraction and normalization.
train_DF=normalize(Xtrain,150)
test_DF =normalize(Xtest,150)

trainDF=pd.DataFrame(train_DF.T)
trainDF['label']=pd.DataFrame(ytrain)
print("When T is equal to 150\n")
print("Head of Train Dataframe\n")
print(trainDF.head())
print("\n")

print("Head of test Dataframe\n")
testDF=pd.DataFrame(test_DF.T)
testDF['label']=pd.DataFrame(ytest)
print(testDF.head())
print("\n")

mu_3=np.mean(trainDF[[0,1]][trainDF['label']==3])
print("The maximum likelihood estimates for mean of 3 is \n",mu_3,"\n")
mu_7=np.mean(trainDF[[0,1]][trainDF['label']==7])
print("The maximum likelihood estimates for mean of 7 is\n",mu_7,"\n")

cov_3=np.cov(trainDF[[0,1]][trainDF['label']==3].T)
print("The maximum likelihood estimates for covariance of 3 is\n",cov_3,"\n")
cov_7=np.cov(trainDF[[0,1]][trainDF['label']==7].T)
print("The maximum likelihood estimates for covariance of 7 is\n",cov_7,"\n")


train_DF=normalize(Xtrain,200)
test_DF =normalize(Xtest,200)
print("When T is equal to 200\n")
trainDF=pd.DataFrame(train_DF.T)
trainDF['label']=pd.DataFrame(ytrain)
print("Head of Train Dataframe")
print(trainDF.head())
print("\n")

print("Head of test Dataframe")
testDF=pd.DataFrame(test_DF.T)
testDF['label']=pd.DataFrame(ytest)
print(testDF.head())
print("\n")

mu_3=np.mean(trainDF[[0,1]][trainDF['label']==3])
print("The maximum likelihood estimates for mean of 3 is \n",mu_3,"\n")
mu_7=np.mean(trainDF[[0,1]][trainDF['label']==7])
print("The maximum likelihood estimates for mean of 7 is\n",mu_7,"\n")

cov_3=np.cov(trainDF[[0,1]][trainDF['label']==3].T)
print("The maximum likelihood estimates for covariance of 3 is\n",cov_3,"\n")
cov_7=np.cov(trainDF[[0,1]][trainDF['label']==7].T)
print("The maximum likelihood estimates for covariance of 7 is\n",cov_7,"\n")


When T is equal to 150

Head of Train Dataframe

          0         1  label
0 -0.312630 -0.051244      3
1  0.965460 -0.847999      3
2  0.516517 -0.815870      3
3  0.297382 -0.489563      3
4  1.906885 -1.441811      3


Head of test Dataframe

          0         1  label
0 -1.174485  1.142590      3
1 -0.444426  0.203891      3
2 -1.184313  1.324994      3
3 -1.922119  2.850372      3
4 -0.139868 -0.286138      3


The maximum likelihood estimates for mean of 3 is 
 0   -0.379520
1    0.358546
dtype: float64 

The maximum likelihood estimates for mean of 7 is
 0    0.371585
1   -0.351050
dtype: float64 

The maximum likelihood estimates for covariance of 3 is
 [[ 0.8437565  -0.91737529]
 [-0.91737529  1.10355626]] 

The maximum likelihood estimates for covariance of 7 is
 [[ 0.87417217 -0.72010492]
 [-0.72010492  0.64980588]] 

When T is equal to 200
Head of Train Dataframe
          0         1  label
0 -0.312630 -0.029591      3
1  0.965460 -0.826533      3
2  0.516517 -0.72544

In [7]:
#Calculate the probability density function using the MLE estimates.
three_distribution = multivariate_normal(mu_3, cov_3)
seven_distribution = multivariate_normal(mu_7, cov_7)

pdf3 = lambda x: three_distribution.pdf(x)
pdf7 = lambda x: seven_distribution.pdf(x)

# Classification

In [8]:
#Function which classifies using minimum error rate classification.
def classify(X, pof3, pof7):
    pof3_x = pdf3(X) * pof3
    pof7_x = pdf7(X) * pof7
    if pof3_x >= pof7_x:
        return 3
    else:
        return 7

In [9]:
#Predicting the train and test results
y_pred_train = [classify(i,0.5,0.5) for i in train_DF.T]
y_pred_test = [classify(i,0.5,0.5) for i in test_DF.T]

In [10]:
#Storing them in our dataframe.
trainDF['Y_pred'] = y_pred_train
testDF['Y_pred'] = y_pred_test

In [12]:
#Calculating the train error rate and test error rate. 
print("The results for the configuration T=150 and P(3)=P(7)==0.5 is as below\n")
train_error_rate = np.mean(trainDF['label'] != trainDF['Y_pred'])
print("The train error rate is ",train_error_rate)
test_error_rate = np.mean(testDF['label'] != testDF['Y_pred'])
print("the test error rate is ",test_error_rate)

The results for the configuration T=150 and P(3)=P(7)==0.5 is as below

The train error rate is  0.28827502597852445
the test error rate is  0.2955647955647956


# Calculating error rates for all configurations

In [19]:
print(" T"," P(3)_and_P(7)","Train_Error_rate"," Test_Error_Rate")
for T in (150,200):
    for P in ((0.5,0.5),(0.3,0.7)):
        train_DF=normalize(Xtrain,T)
        test_DF =normalize(Xtest,T)
        trainDF=pd.DataFrame(train_DF.T)
        trainDF['label']=pd.DataFrame(ytrain)
        testDF=pd.DataFrame(test_DF.T)
        testDF['label']=pd.DataFrame(ytest)
        
        mu_3=np.mean(trainDF[[0,1]][trainDF['label']==3])
        mu_7=np.mean(trainDF[[0,1]][trainDF['label']==7])
        cov_3=np.cov(trainDF[[0,1]][trainDF['label']==3].T)
        cov_7=np.cov(trainDF[[0,1]][trainDF['label']==7].T)
        
        three_distribution = multivariate_normal(mu_3, cov_3)
        seven_distribution = multivariate_normal(mu_7, cov_7)
        pdf3 = lambda x: three_distribution.pdf(x)
        pdf7 = lambda x: seven_distribution.pdf(x)
        
        y_pred_train = [classify(i,P[0],P[1]) for i in train_DF.T]
        y_pred_test = [classify(i,P[0],P[1]) for i in test_DF.T]
        trainDF['Y_pred'] = y_pred_train
        testDF['Y_pred'] = y_pred_test
        train_error_rate = np.mean(trainDF['label'] != trainDF['Y_pred'])
        test_error_rate = np.mean(testDF['label'] != testDF['Y_pred'])
        print(T,"",P,"",train_error_rate,"",test_error_rate)
        

 T  P(3)_and_P(7) Train_Error_rate  Test_Error_Rate
150  (0.5, 0.5)  0.3428299272601316  0.33853083853083854
150  (0.3, 0.7)  0.40656390717007274  0.4078309078309078
200  (0.5, 0.5)  0.28827502597852445  0.2955647955647956
200  (0.3, 0.7)  0.40318669899549703  0.41302841302841303
